## import函式庫並讀取檔案

In [1]:
pip install zhconv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 211 kB 5.1 MB/s 
  Created wheel for zhconv: filename=zhconv-1.4.3-py2.py3-none-any.whl size=208852 sha256=443185cd0c5abc8ed7167202074d27cdffafef771e1b4e75b9214a4caa4719d2
  Stored in directory: /root/.cache/pip/wheels/c8/11/df/c14a55367e9dc3cf3d605f1335d484797fc01c9d61de740bb3
Successfully built zhconv


In [2]:
import requests
from bs4 import BeautifulSoup
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import json 
import time
import random
import threading
from zhconv import convert
import re
from googlesearch import search

# 讀取JSON檔案
with open('/content/Stupid_QA_200.json') as f:
  data = json.load(f)

Mounted at /content/drive


## 資料預處理
將題目中會影響搜尋結果不好的單詞拿掉

In [3]:
def preprocessing(question):
  if '下列' in question:
    question = question.replace('下列', '')
  if '何者' in question:
    question = question.replace('何者', '')
  if '哪個' in question:
    question = question.replace('哪個', '')
  if '哪位' in question:
    question = question.replace('哪位', '')
  if '為' in question:
    question = question.replace('為', '')
  if '請問' in question:
    question = question.replace('請問', '')
  if '?' in question:
    question = question.replace('?', '')
  if '中華民國' in question:
    question = question.replace('中華民國', '台灣')
  return question

## 計算google搜尋網頁中，答案選項分別出現幾次，並回傳出現最多次的選項

In [4]:
def countNum(web_item, question):
  
  # 題目的選項出現的次數
  num_A = 0
  num_B = 0
  num_C = 0
  
  # 尋找網頁中出現過幾次題目的選項
  for i in web_item:
    webText = i.text
    num_A += webText.count( question[2] )
    num_B += webText.count( question[3] )
    num_C += webText.count( question[4] )

  # 回傳出現次數最多的答案選項
  num_answer = [int(num_A), int(num_B), int(num_C)]

  # 如果找不到，將題目的選項轉成簡體字找找看
  if max(num_answer) == 0:  
    num_answer = translated_Simplified_Chinese(web_item, question)

  # 如果找不到，將題目擷取題目的第一句拿去搜尋
  # if max(num_answer) == 0:  
  #   num_answer = search_again_part(question)
    
  # 如果最多次就回傳,找不到答案就猜C
  if max(num_answer) == num_A:
    return "A"
  elif max(num_answer) == num_B:
    return "B"
  elif max(num_answer) == num_C:
    return "C"  
  else:     
    return "C"

In [5]:
def translated_Simplified_Chinese(web_item, question):
  num_A = 0
  num_B = 0
  num_C = 0
  
  for i in web_item:
    webText = i.text
    num_A += webText.count( convert(question[2], 'zh-hans') )
    num_B += webText.count( convert(question[3], 'zh-hans') )
    num_C += webText.count( convert(question[4], 'zh-hans') )
    
  # 回傳出現次數最多的答案選項
  num_answer = [int(num_A), int(num_B), int(num_C)]

  return num_answer

## 擷取題目的第一句

In [6]:
# def extract_part_of_question(question):
#   comma_position = question.find('，')
#   return question[:comma_position]

In [7]:
# def search_again_part(question):
    
#     question = extract_part_of_question(question[1])
#     print("search_again_part question = ", question)

#     # Google 搜尋 URL
#     google_url = 'https://www.google.com.tw/search?q=' + question

#     # 反反爬蟲機制，在每次發請求前讓程式暫停一個隨機的秒數，讓請求頻率降低，且間隔不固定也比較不會被認為是爬蟲
#     time.sleep(random.uniform(5, 10))

#     # 下載 Google 搜尋結果
#     searchResult = requests.get(google_url)
        
#     # 確認是否下載成功
#     if searchResult.status_code == requests.codes.ok:

#       # 以 BeautifulSoup 解析 HTML 原始碼
#       soup = BeautifulSoup(searchResult.text, 'html.parser')

#       # 找到HTML中名為.BNeawe的class
#       web_item = soup.select('.BNeawe')
      
#       # 題目的選項出現的次數
#       num_A = 0
#       num_B = 0
#       num_C = 0
      
#       # 尋找網頁中出現過幾次題目的選項
#       for i in web_item:
#         webText = i.text
#         num_A += webText.count( question[2] )
#         num_B += webText.count( question[3] )
#         num_C += webText.count( question[4] )
#       # 回傳出現次數最多的答案選項
#       num_answer = [int(num_A), int(num_B), int(num_C)]

#       return num_answer      

## 答案

In [8]:
# 200的正確答案
# correct_answer = ['B','A','C','B','A','C','C','C','A','C',
#                   'C','A','A','B','C','C','A','C','B','C',
#                   'B','C','B','C','C','A','B','A','B','C',
#                   'B','C','C','C','B','B','B','C','B','B',
#                   'C','C','B','B','B','A','B','A','A','B',
#                   'C','B','C','B','C','A','B','B','A','C',
#                   'A','A','B','B','A','C','B','C','A','A',
#                   'B','A','B','B','C','C','B','A','B','A',
#                   'B','B','C','A','C','A','A','B','B','C',
#                   'C','A','B','B','C','A','B','A','B','B',
#                   'B','C','A','C','B','A','B','C','A','C',
#                   'A','A','C','A','A','A','C','B','A','C',
#                   'B','C','B','C','B','A','A','C','B','B',
#                   'C','A','B','A','A','C','A','C','A','A',
#                   'C','A','C','B','A','B','B','A','C','A',
#                   'C','C','A','A','A','B','C','A','B','C',
#                   'A','A','C','A','C','A','A','B','C','A',
#                   'C','B','C','C','B','C','A','A','C','C',
#                   'B','C','A','A','B','A','B','B','C','C',
#                   'B','B','C','B','B','C','B','A','C','B']

## 比較搜尋到的答案和正確答案的差異並計算分數



In [9]:
def compare_answer(search_answer, correct_answer):

  wrong_no = []
  # 計算有幾題搜尋結果和正確答案不一樣
  wrong_num = 0
  for i in range(len(search_answer)):
    if search_answer[i][1] != correct_answer[i]:    # 判斷答案是否和正解一樣  
      wrong_num += 1                                # 計算有幾題不一樣
      wrong_no.append(i)

  # 計算分數
  score = (len(search_answer)- wrong_num)/len(search_answer) * 100

  return wrong_num, score, wrong_no

## 爬蟲Google搜尋題目的結果

In [10]:
def search(question, search_answer):

    # Google 搜尋 URL
    google_url = 'https://www.google.com.tw/search?q=' + question[1]

    # 反反爬蟲機制，在每次發請求前讓程式暫停一個隨機的秒數，讓請求頻率降低，且間隔不固定也比較不會被認為是爬蟲
    time.sleep(random.uniform(5, 10))

    # 下載 Google 搜尋結果
    searchResult = requests.get(google_url)
    
    # print("searchResult.status_code = ", searchResult.status_code)
    
    # 確認是否下載成功
    if searchResult.status_code == requests.codes.ok:

      # 以 BeautifulSoup 解析 HTML 原始碼
      soup = BeautifulSoup(searchResult.text, 'html.parser')

      # 找到HTML中名為.BNeawe的class
      web_item = soup.select('.BNeawe')

      # 將爬蟲結果中出現最多次的答案選項加到答案list裡面
      search_answer.append( [question[0], countNum(web_item, question)] )

## 取得題目

In [11]:
def access_question(data):
  question = []
  index = 1
  for question_item in data:
    question.append([index, preprocessing( question_item['Question']), question_item['A'], question_item['B'], question_item['C']])
    index += 1
  return question

## 主程式

In [12]:
# 放答案的list
search_answer = []

# threading的list
threads = []

# 取得題目
question = access_question(data)

i = 0
for index in question:
  # 設定子執行緒並加入要執行的執行緒的list中
  threads.append( threading.Thread(target = search, args = (index, search_answer,  )))
  # 執行此執行緒
  threads[i].start()
  i += 1

for j in range(len(data)):
  # 等待這個執行緒工作完畢
  threads[j].join()

# 將答案按照題目順序排序好
search_answer.sort()

res = []
# 將答案加到結果的list中(不要題號)
for i in search_answer:
  res.append(i[1])
 
# 印出答案
print(res)

['C', 'A', 'C', 'B', 'A', 'C', 'A', 'C', 'C', 'C', 'B', 'A', 'A', 'B', 'C', 'A', 'A', 'C', 'B', 'C', 'B', 'C', 'C', 'A', 'C', 'A', 'B', 'A', 'B', 'A', 'B', 'C', 'C', 'C', 'B', 'B', 'B', 'C', 'B', 'B', 'C', 'C', 'B', 'A', 'B', 'A', 'B', 'A', 'A', 'B', 'C', 'B', 'C', 'B', 'C', 'A', 'B', 'B', 'A', 'C', 'A', 'A', 'B', 'B', 'A', 'C', 'B', 'C', 'A', 'C', 'B', 'A', 'B', 'B', 'C', 'C', 'B', 'A', 'B', 'A', 'A', 'A', 'C', 'A', 'C', 'A', 'A', 'B', 'B', 'C', 'C', 'A', 'B', 'B', 'C', 'A', 'B', 'A', 'A', 'B', 'C', 'C', 'A', 'C', 'B', 'A', 'B', 'C', 'A', 'C', 'A', 'C', 'C', 'A', 'A', 'A', 'C', 'B', 'A', 'C', 'B', 'C', 'B', 'C', 'A', 'A', 'A', 'C', 'B', 'B', 'C', 'A', 'B', 'A', 'A', 'C', 'A', 'A', 'A', 'A', 'C', 'A', 'C', 'B', 'A', 'B', 'B', 'A', 'C', 'A', 'C', 'C', 'A', 'A', 'A', 'B', 'C', 'A', 'B', 'C', 'A', 'A', 'C', 'C', 'C', 'A', 'A', 'B', 'C', 'A', 'C', 'B', 'C', 'C', 'B', 'C', 'A', 'A', 'C', 'C', 'B', 'C', 'A', 'A', 'B', 'A', 'B', 'B', 'C', 'C', 'B', 'B', 'C', 'A', 'B', 'C', 'B', 'A', 'C', 'B']

## 印出爬蟲結果中出現最多次的答案選項

In [13]:
# # 錯誤的題數和分數
# wrong_num, score, wrong_no = compare_answer(search_answer, correct_answer)
# print("Wrong_num:", wrong_num)
# print("Score:", score)

## 印出第幾題是錯的

In [14]:
# def print_Wrong_No(wrong_no):
#   for no in wrong_no:                 # 遍歷錯的題號的list
#     for i in range(len(question)):    # 遍歷所有題目
#       if int(no) == int(i):           # 看題目有沒有出現在錯的題號的list
#         print("第", no+1, "題:", question[i][1]) # 印出答案錯誤的題目
#         print(" A", question[i][2])   # 印出答案錯誤的答案
#         print(" B", question[i][3])   # 印出答案錯誤的答案
#         print(" C", question[i][4])   # 印出答案錯誤的答案

In [15]:
# 印出第幾題是錯的
# print_Wrong_No(wrong_no)